In [ ]:
# Masukan seluruh library

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import random
import shutil
from google.colab import drive
from zipfile import ZipFile

import io
import scipy.misc
import numpy as np
import six
import time
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
#Import Goolge Drive ke Collab
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [ ]:
#UnZip data yang ada di goolge drive
with ZipFile('/content/gdrive/MyDrive/data/raw_data.zip', 'r') as zipObj:
  zipObj.extractall('/content/gdrive/MyDrive/data')

In [ ]:
# 1. Buat direktori data/split_data dan jalankan perintah split train test
raw_data_path = "/content/gdrive/MyDrive/data/raw_data/images"
train_data_path = "/content/gdrive/MyDrive/data/split_data/train"
test_data_path = "/content/gdrive/MyDrive/data/split_data/test"

if not os.path.exists("/content/gdrive/MyDrive/data/split_data/"):
  os.mkdir("/content/gdrive/MyDrive/data/split_data/")

try:
  shutil.rmtree(train_data_path)
  shutil.rmtree(test_data_path)
except:
  pass

os.mkdir(train_data_path)
os.mkdir(test_data_path)

total_files = glob.glob(raw_data_path + "/*.jpg")
print("Sample file = ", total_files[0])
print("Total file = ", len(total_files))

indices = list(range(len(total_files)))
train_indices = random.sample(indices, k = int(len(total_files)*0.9))
print("Total train file = ", len(train_indices))

for i in indices:

    if (i in train_indices):
        # Copy to train
        shutil.copy(total_files[i], train_data_path + total_files[i].replace(raw_data_path,""))
        shutil.copy(total_files[i].replace("images","labels").replace(".jpg",".xml"),
                    train_data_path + total_files[i].replace(raw_data_path, "").replace(".jpg",".xml"))
    else:
        # Copy to test
        shutil.copy(total_files[i], test_data_path + total_files[i].replace(raw_data_path, ""))
        shutil.copy(total_files[i].replace("images", "labels").replace(".jpg", ".xml"),
                    test_data_path + total_files[i].replace(raw_data_path, "").replace(".jpg",".xml"))



Sample file =  /content/gdrive/MyDrive/data/raw_data/images/10_jpg.rf.2458551329cd7f49d79564e174a2dc8a.jpg
Total file =  1476
Total train file =  1328


In [ ]:
#3.Konversi label XML ke CSV. File CSV disimpan di folder split_data
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(os.path.join(path, '*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bndbox = member.find('bndbox')
            if bndbox is not None and all(bndbox.find(coord) is not None for coord in ['xmin', 'ymin', 'xmax', 'ymax']):
                value = (root.find('filename').text,
                         int(root.find('size').find('width').text),
                         int(root.find('size').find('height').text),
                         member.find('name').text,
                         int(bndbox.find('xmin').text),
                         int(bndbox.find('ymin').text),
                         int(bndbox.find('xmax').text),
                         int(bndbox.find('ymax').text))
                xml_list.append(value)
            else:
                print(f"Skipping {xml_file}, as it doesn't match the expected structure.")
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

for directory in ['train','test']:
        image_path = os.path.join('/content/gdrive/MyDrive/data/split_data/{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('/content/gdrive/MyDrive/data/split_data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')

Successfully converted xml to csv.
Successfully converted xml to csv.


In [ ]:
# 4. Unduh TF Object Detection APi ke dalam folder model
%cd /content/gdrive/MyDrive
!git clone https://github.com/tensorflow/models.git

/content/gdrive/MyDrive
Cloning into 'models'...
remote: Enumerating objects: 97295, done.
remote: Counting objects: 100% (575/575), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 97295 (delta 341), reused 474 (delta 283), pack-reused 96720
Receiving objects: 100% (97295/97295), 612.92 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (70753/70753), done.
Updating files: 100% (3705/3705), done.


In [ ]:
#5. Kompilasi proto -
# Protocol Buffers (Protobuf) adalah format data lintas platform sumber terbuka dan gratis yang digunakan untuk membuat serial data terstruktur.
%cd /content/gdrive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/MyDrive/models/research


In [ ]:
#6. Instal API
%cd /content/drive/MyDrive/models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

[Errno 2] No such file or directory: '/content/drive/MyDrive/models/research'
/content/gdrive/MyDrive/models/research
Processing /content/gdrive/MyDrive/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 

In [ ]:
# 7. Test API
!python object_detection/builders/model_builder_tf2_test.py

2024-06-02 14:54:22.008959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 14:54:23.332767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/models/research/object_detection/builders/model_builder_tf2_test.py", line 24, in <module>
    from object_detection.builders import model_builder
  File "/usr/local/lib/python3.10/dist-packages/object_detection/builders/model_builder.py", line 26, in <module>
    from object_detection.builders import hyperparams_builder
  File "/usr/local/lib/python3.10/dist-packages/object_detection/builders/hyperparams_builder.py", line 27, in <module>
    from object_detection.core import freezabl

In [ ]:
# 8. CSV to TFRecord. With help from MiAI git hub
%cd /content/gdrive/MyDrive/models/research
!git clone https://github.com/thangnch/MiAI_SSD_TF2
!cp MiAI_SSD_TF2/generate_tfrecord.py .
!cp MiAI_SSD_TF2/label_map.txt /content/gdrive/MyDrive/data

if not os.path.exists("/content/gdrive/MyDrive/data/tfrecord_data/"):
  os.mkdir("/content/gdrive/MyDrive/data/tfrecord_data/")

!python generate_tfrecord.py --image_dir=/content/gdrive/MyDrive/data/split_data/train --csv_input=/content/gdrive/MyDrive/data/split_data/train_labels.csv --output_path=/content/gdrive/MyDrive/data/tfrecord_data/train.record
!python generate_tfrecord.py --image_dir=/content/gdrive/MyDrive/data/split_data/test --csv_input=/content/gdrive/MyDrive/data/split_data/test_labels.csv --output_path=/content/gdrive/MyDrive/data/tfrecord_data/test.record


/content/gdrive/MyDrive/models/research
fatal: destination path 'MiAI_SSD_TF2' already exists and is not an empty directory.
2024-06-02 15:37:45.740604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 15:37:46.940273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-02 15:37:48.778681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-02 15:37:48.851120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] 

In [ ]:
# 9. Download pretrain and config
if not os.path.exists("/content/gdrive/MyDrive/pretrained"):
  os.mkdir("/content/gdrive/MyDrive/pretrained")

%cd /content/gdrive/MyDrive/pretrained
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
# Unzip
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# Config pipeline.config, label_map.txt if needed

/content/gdrive/MyDrive/pretrained
--2024-06-02 15:38:27--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.4.207, 172.253.118.207, 74.125.200.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.4.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.2’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  9.91MB/s    in 2.0s    

2024-06-02 15:38:31 (9.91 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.2’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/chec

In [ ]:
# Upload label file object-detection.txt to data folder
%cd /content/gdrive/MyDrive/models
!pip install -r /content/gdrive/MyDrive/models/research/MiAI_SSD_TF2/setup.txt
!pip install --upgrade opencv-python

/content/gdrive/MyDrive/models


In [ ]:
os.path.exists('/content/gdrive/MyDrive/models/research')
os.path.exists('/content/gdrive/MyDrive/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config')
os.path.exists('/content/gdrive/MyDrive/output_model')

print(os.path.exists('/content/gdrive/MyDrive/models/research'))
print(os.path.exists('/content/gdrive/MyDrive/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'))
print(os.path.exists('/content/gdrive/MyDrive/output_model'))


True
True
False


In [ ]:
# Train
%cd /content/gdrive/MyDrive/models/research
%cp /content/gdrive/MyDrive/models/research/object_detection/model_main_tf2.py .

/content/gdrive/MyDrive/models/research
2024-06-02 15:41:52.152203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 15:41:53.635207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/models/research/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "/content/gdrive/MyDrive/models/research/object_detection/model_lib_v2.py", line 30, in <module>
    from object_detection import inputs
  File "/content/gdrive/MyDrive/models/research/object_detection/inputs.py", line 24, in <module>
    from tensorflow.compat.v1 import estimator as tf_estimator
ImportError: cannot import name 'estimato

In [ ]:
#train
!python model_main_tf2.py \
--pipeline_config_path=/content/gdrive/MyDrive/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
--model_dir=/content/gdrive/MyDrive/output_model --alsologtostderr --num_train_steps=1000

In [ ]:
# Export model
%cd /content/gdrive/MyDrive/models/research
%cp /content/gdrive/MyDrive/models/research/object_detection/exporter_main_v2.py .

!python exporter_main_v2.py \
--trained_checkpoint_dir=/content/gdrive/MyDrive/output_model \
--pipeline_config_path=/content/gdrive/MyDrive/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
--output_directory=/content/gdrive/MyDrive/export_model

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

#Load model
tf.keras.backend.clear_session()
model = tf.saved_model.load("/content/gdrive/MyDrive/export_model/saved_model")

In [ ]:
# Các hàm inference

import cv2
def run_inference_for_single_image(model, image):

  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

def load_image_into_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
%cd /content/gdrive/MyDrive/models/research
category_index = label_map_util.create_category_index_from_labelmap("/content/gdrive/MyDrive/data/label_map.txt", use_display_name=True)

image_path = '/content/gdrive/MyDrive/data/split_data/train/2.jpg'
image_np = load_image_into_numpy_array(image_path)
print("Done load image ")
image_np = cv2.resize(image_np, dsize=None, fx=0.2, fy=0.2)
output_dict = run_inference_for_single_image(model, image_np)
print("Done inference")
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=8)
print("Done draw on image ")
display(Image.fromarray(image_np))